In [32]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [33]:
df = pd.read_csv('data/cfb_data.csv')
df.describe().round(2)

,id,year,week,postseason,id_home_team,id_away_team,points_home,points_away,completed,conference_game,home_team_id,away_team_id
count,97109.0,97109.00,97109.00,97109.00,97109.00,97109.00,96744.00,96744.00,97109.00,97109.00,97109.00,97109.00
mean,48555.0,1972.60,6.83,0.02,49423.46,77702.19,24.22,16.91,1.00,0.53,49423.46,77702.19
std,28033.1,38.14,3.50,0.13,214397.67,265522.78,16.46,14.17,0.03,0.50,214397.67,265522.78
min,1.0,1869.00,1.00,0.00,2.00,2.00,0.00,0.00,0.00,0.00,2.00,2.00
25%,24278.0,1941.00,4.00,0.00,142.00,154.00,12.00,6.00,1.00,0.00,142.00,154.00
50%,48555.0,1978.00,7.00,0.00,252.00,295.00,22.00,14.00,1.00,1.00,252.00,295.00
75%,72832.0,2008.00,10.00,0.00,2348.00,2460.00,34.00,26.00,1.00,1.00,2348.00,2460.00
max,97109.0,2022.00,20.00,1.00,1000929.00,1000932.00,222.00,158.00,1.00,1.00,1000929.00,1000932.00


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97109 entries, 0 to 97108
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     97109 non-null  int64  
 1   year                   97109 non-null  int64  
 2   week                   97109 non-null  int64  
 3   postseason             97109 non-null  int64  
 4   id_home_team           97109 non-null  int64  
 5   id_away_team           97109 non-null  int64  
 6   points_home            96744 non-null  float64
 7   points_away            96744 non-null  float64
 8   completed              97109 non-null  int64  
 9   conference_game        97109 non-null  int64  
 10  home_team_id           97109 non-null  int64  
 11  home_team_full_name    97109 non-null  object 
 12  home_team_school_name  97109 non-null  object 
 13  home_team_mascot       96055 non-null  object 
 14  away_team_id           97109 non-null  int64  
 15  aw

In [35]:
empty_rows = df.isnull().any(axis=1).sum()
print(empty_rows)

4566


In [36]:
df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97109 entries, 0 to 97108
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     97109 non-null  int64  
 1   year                   97109 non-null  int64  
 2   week                   97109 non-null  int64  
 3   postseason             97109 non-null  int64  
 4   id_home_team           97109 non-null  int64  
 5   id_away_team           97109 non-null  int64  
 6   points_home            96744 non-null  float64
 7   points_away            96744 non-null  float64
 8   completed              97109 non-null  int64  
 9   conference_game        97109 non-null  int64  
 10  home_team_id           97109 non-null  int64  
 11  home_team_full_name    97109 non-null  object 
 12  home_team_school_name  97109 non-null  object 
 13  home_team_mascot       96055 non-null  object 
 14  away_team_id           97109 non-null  int64  
 15  aw

In [37]:
cleaned_df = df.drop_duplicates()
empty_rows = cleaned_df.isnull().any(axis=1).sum()
print(cleaned_df)

          id  year  week  postseason  id_home_team  id_away_team  points_home  \
0          1  1869     1           0           164           163          6.0   
1          2  1869     2           0           163           164          8.0   
2          3  1870     1           0           164           171          6.0   
3          4  1870     2           0           163           164          6.0   
4          5  1872     1           0           164           171          0.0   
...      ...   ...   ...         ...           ...           ...          ...   
97104  97105  2022     1           1          2509            99          7.0   
97105  97106  2022     1           1            30          2655         45.0   
97106  97107  2022     1           1           254           213         21.0   
97107  97108  2022     1           1          2571          2449         45.0   
97108  97109  2022     1           1            61          2628         65.0   

       points_away  complet

In [38]:
print(empty_rows)

4566


In [39]:
invalid_df = pd.DataFrame()
null_row_mask = df.isnull().any(axis=1)
rows_with_null = df[null_row_mask]
invalid_df = pd.concat([invalid_df, rows_with_null], ignore_index=True)
print(invalid_df)

         id  year  week  postseason  id_home_team  id_away_team  points_home  \
0        13  1873     3           0       1000055       1000066          6.0   
1        17  1873     6           0            43       1000067          2.0   
2        30  1875     3           0       1000068           108          0.0   
3        31  1875     3           0       1000055       1000066          6.0   
4        36  1875     4           0       1000066           171          0.0   
...     ...   ...   ...         ...           ...           ...          ...   
4561  97036  2022    14           0          3071           175          NaN   
4562  97056  2022    15           0           426          2685          NaN   
4563  97057  2022    15           0          2222        110242          NaN   
4564  97060  2022    15           0          3071          2371          NaN   
4565  97061  2022    15           0          2146          2974          NaN   

      points_away  completed  conferenc

In [ ]:
cleaned_df['home_team_mascot'].fillna('N/A', inplace=True)
cleaned_df['away_team_mascot'].fillna('N/A', inplace=True)
cleaned_df.dropna(inplace=True)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96744 entries, 0 to 97108
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     96744 non-null  int64  
 1   year                   96744 non-null  int64  
 2   week                   96744 non-null  int64  
 3   postseason             96744 non-null  int64  
 4   id_home_team           96744 non-null  int64  
 5   id_away_team           96744 non-null  int64  
 6   points_home            96744 non-null  float64
 7   points_away            96744 non-null  float64
 8   completed              96744 non-null  int64  
 9   conference_game        96744 non-null  int64  
 10  home_team_id           96744 non-null  int64  
 11  home_team_full_name    96744 non-null  object 
 12  home_team_school_name  96744 non-null  object 
 13  home_team_mascot       96744 non-null  object 
 14  away_team_id           96744 non-null  int64  
 15  away_te

In [ ]:
# cleaned_df.to_csv('data/cleaned_cfb_data.csv', index=False)
# cleaned_df.to_json('data/cleaned_cfb_data.json', orient='records', indent=4)